# How would Height of the basketball player affect the force it takes to make a free throw?

In [99]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

Fm = S (w × v)

In [100]:
m = UNITS.meter
s = UNITS.second
kg = UNITS.kilogram
degree = UNITS.degree
rad = UNITS.radian

<Unit('radian')>

In [101]:

params = Params(x = 0 * m,                   # x coordinate of the basketball
                y = 0 * m,                  # y coordinate of the basketball
                omega = 0 * rad / s ** 2,    # velocity
                rho = 1.2 * kg / m ** 3,     # mass density of the fluid
                C_d = 0.54,                  # drag coefficient of a basketball
                x_dist = 4.9 * m,            # horizontal distance of player to net
                y_dist = 3.05 * m,           # distance of the rim from ground
                v = 0 * m / s,               # velocity of the basketball
                S = 0.47,                    # air resistance coefficient of spheres
                g = 9.8 * m/s**2,            # gravity
                angle = 52 * degree,               # launch angle of the basketball
                mass = 0.625 * kg,           # mass of the basketball
                diameter = 0.23 * m,          # diameter of the basketball
                t_end = 10 * s)

,values
x,0 meter
y,0 meter
omega,0.0 radian / second ** 2
rho,1.2 kilogram / meter ** 3
C_d,0.54
x_dist,4.9 meter
y_dist,3.05 meter
v,0.0 meter / second
S,0.47
g,9.8 meter / second ** 2


In [102]:
def make_system(params, v_0, height):
    """Make a system object.
    
    params: 
    
    returns: System object
    """
    unpack(params)
     # convert angle to degrees
    theta = np.deg2rad(angle)
    
    # compute x and y components of velocity
    vx, vy = pol2cart(theta, v_0)

    init = State(x = x, y = height, vx = vx, vy = vy)

    # compute area from diameter
    area = np.pi * (diameter/2)**2
    
    return System(params, init=init, area = area)

In [103]:
system = make_system(params, 0 * m / s, 1.8 * m)

,values
x,0 meter
y,0 meter
omega,0.0 radian / second ** 2
rho,1.2 kilogram / meter ** 3
C_d,0.54
x_dist,4.9 meter
y_dist,3.05 meter
v,0.0 meter / second
S,0.47
g,9.8 meter / second ** 2


In [104]:
def drag_force(V, system):
    """Computes drag force in the opposite direction of `V`.
    
    V: velocity
    system: System object with rho, C_d, area
    
    returns: Vector drag force
    """
    unpack(system)
    mag = -rho * V.mag**2 * C_d * area / 2
    direction = V.hat()
    f_drag = mag * direction
    return f_drag

In [105]:
def slope_func(state, t, system):
    """Computes derivatives of the state variables.
    
    state: State (x, y, x velocity, y velocity)
    t: time
    system: System object with g, rho, C_d, area, mass
    
    returns: sequence (vx, vy, ax, ay)
    """
    x, y, vx, vy = state
    unpack(system)

    V = Vector(vx, vy)    
    a_drag = drag_force(V, system) / mass
    a_grav = Vector(0, -g)
    
    a = a_grav + a_drag
    
    return vx, vy, a.x, a.y

In [106]:
def event_func(state, t, system):
    """Stop when the y coordinate is 0.
    
    state: State object
    t: time
    system: System object
    
    returns: y coordinate
    """
    x, y, vx, vy = state
    return x - 4.9

In [132]:
def height_func(v_0, height):
    system = make_system(params, v_0 = v_0, height = height)
    results, details = run_ode_solver(system, slope_func, events=event_func)
    return Vector(get_last_value(results.x) * m, get_last_value(results.y) * m)

In [134]:
height_func(20 * m /s, 1.8*  m)

<Quantity([4.9        7.20318324], 'meter')>

In [161]:
velocities = linspace(8, 10, 21)
heights = linspace(1.6, 2.3, 8)
sweep_h = SweepSeries()

print(heights)
for h in heights:
    sweep_v = SweepSeries()
    for vel in velocities:
        vector = height_func(vel * m / s, h * m)
        print(vector.y)
        if(vector.y < 3.165 * m and vector.y > 2.935 * m):
            sweep_v[vel] = vector
    sweep_h[h] = sweep_v
sweep_h

[1.6 1.7 1.8 1.9 2.  2.1 2.2 2.3]
2.5428448902849645 meter


ValueError: Cannot compare Quantity and <class 'pint.quantity.build_quantity_class.<locals>.Quantity'>

In [170]:
vv = Vector(3, 2.542) * m
dd = Vector(0, 3.165) * m

<Quantity([0.    3.165], 'meter')>

In [173]:
 3.165 * m > vv.y

ValueError: Cannot compare Quantity and <class 'pint.quantity.build_quantity_class.<locals>.Quantity'>

In [171]:
vv.y > dd.y

False